In [1]:
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup
import webbrowser
import requests
import lxml.html
import re
from selenium import webdriver
import dart_fss as dart
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException #엘리먼트가 동일하지 않으므로 예외처리

In [2]:
import time

In [2]:
#!pip install selenium

     |████████████████████████████████| 911kB 424kB/s eta 0:00:01


In [3]:
path = "./chromedriver"
driver = webdriver.Chrome(path)

In [5]:
df_rcp = pd.read_csv("./data/company_info.csv",encoding="utf-8-sig")

In [6]:
df_rcp

,Unnamed: 0,company_code,company_name,rcp_no
0,0,155660,DSR,20190329003198
1,1,155660,DSR,20180330002370
2,2,155660,DSR,20170330001011
3,3,155660,DSR,20160330001425
4,4,155660,DSR,20150330000935
...,...,...,...,...
18976,18976,202960,판도라티비,20150925000267
18977,18977,202960,판도라티비,20150925000262
18978,18978,202960,판도라티비,20150601000250
18979,18979,202960,판도라티비,20150331001668


In [7]:
df_rcp["report_6"] = 0 #6번째 레포트와 8번째 레포트 불러올 칼럼 만들기
df_rcp["report_8"] = 0

In [28]:
df_rcp_new = df_rcp[df_rcp["rcp_no"]!=0] #사업 보고서 번호가 있는 것만

In [29]:
df_rcp_new

,Unnamed: 0,company_code,company_name,rcp_no,report_6,report_8
0,0,155660,DSR,20190329003198,0,0
1,1,155660,DSR,20180330002370,0,0
2,2,155660,DSR,20170330001011,0,0
3,3,155660,DSR,20160330001425,0,0
4,4,155660,DSR,20150330000935,0,0
...,...,...,...,...,...,...
18976,18976,202960,판도라티비,20150925000267,0,0
18977,18977,202960,판도라티비,20150925000262,0,0
18978,18978,202960,판도라티비,20150601000250,0,0
18979,18979,202960,판도라티비,20150331001668,0,0


In [34]:
df_rcp_new = df_rcp_new.reset_index() #인덱스를 초기화하지 않으면 이가 안 맞음

In [38]:
df_rcp_new.drop("Unnamed: 0",1,inplace=True)

In [40]:
df_rcp_new.tail()

,level_0,company_code,company_name,rcp_no,report_6,report_8
2572,7020,214610,미코바이오메드,20160330002986,0,0
2573,7021,214610,미코바이오메드,20150626000496,0,0
2574,7022,199290,바이오프로테크,20190401004021,0,0
2575,7023,199290,바이오프로테크,20180402003599,0,0
2576,7024,199290,바이오프로테크,20170331005329,0,0


In [19]:
rcp_no = "20150601000250" #테스트
url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo="+rcp_no+"&eleId=20"
#req = requests.get(url_business)
#tree = lxml.html.fromstring(req.text)
#onclick = tree.xpath('//*[@id="north"]/div[2]/ul/li[1]/a')[0].attrib['onclick']
#onclick = tree.xpath('//*[@id="west-panel"]/div[2]/div/ul/div/li[8]/div/a')[0].attrib['href']
#onclick = tree.xpath('//*[@id="ext-gen10"]/div/li[8]/div/a')[0].attrib['href']
#pattern = re.compile("^openPdfDownload\('\d+',\s*'(\d+)'\)")
#dcm_no = pattern.search(onclick).group(1)

In [20]:
driver.get(url)

In [41]:
k = 0
for rcp_no in df_rcp_new["rcp_no"]:
    str_rcp_no = str(rcp_no)
    url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo="+str_rcp_no+"&eleId=20"
    driver.get(url)
    try:
        mod_str = driver.find_element_by_xpath('//*[@id="ext-gen10"]/div/li[1]/div/a/span').text
        if (mod_str.find("(보고)")>0): #정정신고의 경우
            driver.find_element_by_xpath('//*[@id="ext-gen10"]/div/li[10]/div/a').click() #6번 클릭
            ifrm6_url = driver.find_element_by_id("ifrm").get_attribute("src")
            df_rcp_new.loc[k,"report_6"] = ifrm6_url
            time.sleep(1)
            driver.find_element_by_xpath('//*[@id="ext-gen10"]/div/li[12]/div/a').click() #8번 클릭
            ifrm8_url = driver.find_element_by_id("ifrm").get_attribute("src")
            df_rcp_new.loc[k,"report_8"] = ifrm8_url
        else:        
            driver.find_element_by_xpath('//*[@id="ext-gen10"]/div/li[8]/div/a').click() #6번 클릭
            ifrm6_url = driver.find_element_by_id("ifrm").get_attribute("src")
            df_rcp_new.loc[k,"report_6"] = ifrm6_url
            time.sleep(1)
            driver.find_element_by_xpath('//*[@id="ext-gen10"]/div/li[10]/div/a').click() #8번 클릭
            ifrm8_url = driver.find_element_by_id("ifrm").get_attribute("src")
            df_rcp_new.loc[k,"report_8"] = ifrm8_url
    except NoSuchElementException:
        df_rcp_new.loc[k,"report_6"] = "modified"
        df_rcp_new.loc[k,"report_8"] = "modified"
    k += 1
    time.sleep(1)

In [48]:
df_rcp_new[240:]

,index,company_code,company_name,rcp_no,report_6,report_8
240,240,1520,동양,20160330004390,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
241,241,1520,동양,20150331004144,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
242,242,1520,동양,20140530001445,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
243,243,1520,동양,20140331003678,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
244,244,1520,동양,20130329002662,0,0
...,...,...,...,...,...,...
8325,8325,131220,대한과학,20150331003454,0,0
8326,8326,131220,대한과학,20140530001062,0,0
8327,8327,131220,대한과학,20140328001394,0,0
8328,8328,131220,대한과학,20130401000904,0,0


In [40]:
df_rcp_new.drop("Unnamed: 0",1,inplace=True)

In [34]:
df_rcp_new

,company_code,company_name,rcp_no,report_6,report_8
0,155660.0,DSR,2.019033e+13,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
1,155660.0,DSR,2.018033e+13,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
2,155660.0,DSR,2.017033e+13,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
3,155660.0,DSR,2.016033e+13,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
4,155660.0,DSR,2.015033e+13,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
...,...,...,...,...,...
1760,NaN,NaN,NaN,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
1761,NaN,NaN,NaN,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
1863,NaN,NaN,NaN,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
1973,NaN,NaN,NaN,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...


In [28]:
df_rcp_new.to_csv("crawl_report6_8_2.csv",encoding="utf-8-sig")

In [5]:
driver.close()